
Algorytm mrówkowy

Biblioteki

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

Pobór danych

In [14]:
file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/wera6/Downloads/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/wera6/Downloads/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/wera6/Downloads/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

Algorytm

In [15]:


def ant_colony_optimization(dist_matrix, num_ants=10, num_iterations=100, alpha=1, beta=2, rho=0.5, Q=100):
    start_time = time.time()
    num_cities = len(dist_matrix)
    pheromone = np.ones((num_cities, num_cities)) / num_cities  # Inicjalizacja feromonów
    best_path = None
    best_path_length = float('inf')

    def compute_path_length(path):
        return sum(dist_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1)) + dist_matrix[path[-1]][path[0]]

    def select_next_city(visited, current_city):
        probabilities = []
        for next_city in range(num_cities):
            if next_city not in visited:
                tau = pheromone[current_city][next_city] ** alpha
                eta = (1 / dist_matrix[current_city][next_city]) ** beta
                probabilities.append(tau * eta)
            else:
                probabilities.append(0)
        total = sum(probabilities)
        probabilities = [p / total for p in probabilities]
        return random.choices(range(num_cities), probabilities)[0]

    for iteration in range(num_iterations):
        all_paths = []
        all_path_lengths = []
        
        for ant in range(num_ants):
            path = []
            visited = set()
            current_city = random.randint(0, num_cities - 1)
            path.append(current_city)
            visited.add(current_city)
            
            while len(visited) < num_cities:
                next_city = select_next_city(visited, current_city)
                path.append(next_city)
                visited.add(next_city)
                current_city = next_city

            path_length = compute_path_length(path)
            all_paths.append(path)
            all_path_lengths.append(path_length)

            if path_length < best_path_length:
                best_path_length = path_length
                best_path = path

        # Aktualizacja feromonów
        pheromone *= (1 - rho)  # Parowanie
        for path, length in zip(all_paths, all_path_lengths):
            for i in range(len(path)):
                pheromone[path[i]][path[(i + 1) % num_cities]] += Q / length


    
    total_time = time.time() - start_time

    best_path = [city + 1 for city in best_path]


    return best_path, best_path_length, total_time

# Przykład użycia
# dist_matrix = np.array([
#     [0, 10, 15, 20],
#     [10, 0, 35, 25],
#     [15, 35, 0, 30],
#     [20, 25, 30, 0]
# ])

# best_path, best_length = ant_colony_optimization(dist_matrix)
# print(f"Najlepsza ścieżka: {best_path}")
# print(f"Długość ścieżki: {best_length}")


Generowanie wyników

In [16]:
classic_p_1, classic_c_1, classic_t_1 = ant_colony_optimization(data1)
classic_p_2, classic_c_2, classic_t_2 = ant_colony_optimization(data2)
classic_p_3, classic_c_3, classic_t_3 = ant_colony_optimization(data3)

In [17]:
W1 =  {
    "Długość ścieżki": [classic_c_1,classic_c_2,classic_c_3 ],
    "Ścieżka": [classic_p_1,classic_p_2,classic_p_3 ],
    "Czas": [classic_t_1,classic_t_2,classic_t_3 ]

}

m1 = pd.DataFrame(data = W1)

Badanie wpływu parametru liczba mrówek

In [ ]:


def test_ants_effect(data, dataset_name):
    results = []
    num_cities = len(data)
    for num_ants in range(max(1, int(0.5 * num_cities)), int(2 * num_cities) + 1, 20):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=num_ants, num_iterations=100, alpha=1, beta=2, rho=0.5, Q=100)
        results.append({"PARAMETR": num_ants, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_ants_effect(data1, "DATA1")
df2 = test_ants_effect(data2, "DATA2")
df3 = test_ants_effect(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
ants = pd.concat([df1, df2, df3], ignore_index=True)





Badanie wpływu num_itteration

In [ ]:
def test_itteration(data, dataset_name):
    results = []
   
    for num_iterations in range(50, 500, 50):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants= 10, num_iterations= num_iterations, alpha=1, beta=2, rho=0.5, Q=100)
        results.append({"PARAMETR": num_iterations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_itteration(data1, "DATA1")
df2 = test_itteration(data2, "DATA2")
df3 = test_itteration(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
itt = pd.concat([df1, df2, df3], ignore_index=True)


Badanie wpływu parametru alpha

In [ ]:
def test_alpha(data, dataset_name):
    results = []
   
    for alpha in range(0, 5, 1):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants= 10, num_iterations= 100, alpha=alpha, beta=2, rho=0.5, Q=100)
        results.append({"PARAMETR": alpha, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_alpha(data1, "DATA1")
df2 = test_alpha(data2, "DATA2")
df3 = test_alpha(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
al = pd.concat([df1, df2, df3], ignore_index=True)

Badanie wpływu parametru beta

In [ ]:
def test_beta(data, dataset_name):
    results = []
   
    for beta in range(1, 10, 1):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants= 10, num_iterations= 100, alpha=1, beta=beta, rho=0.5, Q=100)
        results.append({"PARAMETR": beta, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_beta(data1, "DATA1")
df2 = test_beta(data2, "DATA2")
df3 = test_beta(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
bet = pd.concat([df1, df2, df3], ignore_index=True)

Badanie parametru rho


In [ ]:
def test_rho(data, dataset_name):
    results = []
   
    for rho in np.linspace(0.1, 0.9, 9):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants= 10, num_iterations= 100, alpha=1, beta=2, rho=rho, Q=100)
        results.append({"PARAMETR": rho, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 =test_rho(data1, "DATA1")
df2 =test_rho(data2, "DATA2")
df3 = test_rho(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
rho = pd.concat([df1, df2, df3], ignore_index=True)

Badanie wpływu parametru  Q

In [ ]:
def test_qu(data, dataset_name):
    results = []
   
    for Qu in range(0, 500, 50):  # Liczba mrówek w krokach co 5
        best_path, best_cost, best_time = ant_colony_optimization(data, num_ants= 10, num_iterations= 100, alpha=1, beta=2, rho=0.5, Q=Qu)
        results.append({"PARAMETR": Qu, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_qu(data1, "DATA1")
df2 = test_qu(data2, "DATA2")
df3 = test_qu(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
qu = pd.concat([df1, df2, df3], ignore_index=True)

Zapis do pliku Excel

In [ ]:
resu = {
    "Klasyk_wyniki": m1,
    "num_ants": ants,
    "max_itt":  itt,
    "alpha": al,
    "beta": bet,
    "rho": rho,
    "QU" : qu
}

# Ścieżka do pliku Excel
file_name = "MRÓWY.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in resu.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")